In [2]:
import pandas as pd
from numpy import arange
import codecs

In [3]:
%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt

In [4]:
import pickle

In [5]:
jeuName="/Users/gilles/Copy/Recherche/Rugby/Videos/CorrespondancesEvActionsComm1-GB.csv"
rythmeName="/Users/gilles/Copy/Recherche/Rugby/CorpusRugby2007/Timeline-Rythme-MT1.csv"
jeuFile=open(jeuName,"r")
jeu=jeuFile.readlines()

In [6]:
timing=[]
for ligne in jeu[1:]:
    elements=ligne.strip().split(";")
    timing.append([elements[0],elements[1],elements[4],elements[5],elements[6]])

In [7]:
#timeLine=pd.DataFrame(index=[float(i)/100 for i in range(6500000)],columns=["début","fin","V1","V2","V3"])
timeLine=pd.DataFrame(columns=["début","fin","événement","action","comment"])

In [8]:
initialTime="00:11:52.4"
decalage=0
def decodeTime(temps):
    (heure,minute,seconde)=temps.split(":")
    result=int(100*(3600*int(heure)+60*int(minute)+float(seconde))-decalage)
    return result

decalage=decodeTime(initialTime)
print decalage, decodeTime(initialTime)

for ligne in timing:
    if ligne[3] and ligne[2]:
        debut=decodeTime(ligne[0])
#        print debut, ligne
        timeLine.loc[debut]=ligne

71240 0


In [9]:
#timeLine

In [10]:
len(timeLine[(timeLine.index<5000) & (timeLine.index>3200) & (timeLine["comment"]!="")])

4

In [11]:
def adapte(valeur):
    return float(valeur*100)

In [12]:
%%time
rythmes={}
vitesses={}
#Tous les rythmes sont ramenés à 5 secondes comme pour Berlin
echelle=5
demiFenetres=[1,1.5,2,2.5,3,3.5,4,4.5,5]
increment=min(min(demiFenetres)*2,1)
for demiFenetre in demiFenetres:
    rythmes[demiFenetre]={}
    vitesses[demiFenetre]={}
    demiFen=adapte(demiFenetre)
    debut=adapte(30)
    fin=adapte(6000)
    step=adapte(increment)
    for i in arange(debut,fin,step):
        nbActions=len(timeLine[(timeLine.index<=i+demiFen) & (timeLine.index>=i-demiFen)])
        rythmes[demiFenetre][float(i)/100]=float(nbActions)/(2*demiFen/100)*echelle
        nbActionsCommentees=len(timeLine[(timeLine.index<=i+demiFen) & (timeLine.index>=i-demiFen) & (timeLine["comment"]!="")])
        vitesses[demiFenetre][float(i)/100]=float(nbActionsCommentees)/(2*demiFen/100)*echelle
        

CPU times: user 32.9 s, sys: 30.1 ms, total: 32.9 s
Wall time: 32.9 s


In [13]:
def calculerRythme(rythmeCalc,timelineRef):
    resultats={}
    elementCle=timelineRef.keys()[0]
    for elementTime in sorted(timelineRef[elementCle]):
        if elementTime < 6000:
            ligneResultat=[]
            for rythme in sorted(timelineRef):
                ligneResultat.append(timelineRef[rythme][elementTime])
            resultats[elementTime]=ligneResultat
            rythmeCalc.loc[int(adapte(elementTime))]=ligneResultat

In [14]:
%%time
timeRythmes=pd.DataFrame(columns=["RJ%.2f"%i for i in demiFenetres])
calculerRythme(timeRythmes,rythmes)

CPU times: user 9.97 s, sys: 79.1 ms, total: 10 s
Wall time: 10.1 s


In [17]:
#timeRythmes

In [18]:
%%time
timeVitesses=pd.DataFrame(columns=["RC%.2f"%i for i in demiFenetres])
calculerRythme(timeVitesses,vitesses)

CPU times: user 10 s, sys: 72.3 ms, total: 10.1 s
Wall time: 10.1 s


In [37]:
timeVitesses

,RC1.00,RC1.50,RC2.00,RC2.50,RC3.00,RC3.50,RC4.00,RC4.50,RC5.00
3000,0.0,0.000000,0.00,0,0.833333,0.714286,0.625,0.555556,0.5
3100,0.0,0.000000,1.25,1,0.833333,0.714286,0.625,0.555556,0.5
3200,2.5,1.666667,1.25,1,0.833333,0.714286,0.625,0.555556,0.5
3300,2.5,1.666667,1.25,1,0.833333,0.714286,0.625,0.555556,0.5
3400,0.0,1.666667,1.25,1,0.833333,0.714286,0.625,0.555556,0.5
3500,0.0,0.000000,0.00,1,0.833333,0.714286,0.625,0.555556,0.5
3600,0.0,0.000000,0.00,0,0.000000,0.714286,0.625,0.555556,1.0
3700,0.0,0.000000,0.00,0,0.000000,0.000000,0.625,1.111111,1.0
3800,0.0,0.000000,0.00,0,0.833333,0.714286,0.625,0.555556,0.5
3900,0.0,0.000000,1.25,1,0.833333,0.714286,0.625,0.555556,0.5


In [23]:
timeRythmes

,R1.00,R1.50,R2.00,R2.50,R3.00,R3.50,R4.00,R4.50,R5.00
3000,0.0,0.000000,0.00,0,0.833333,0.714286,0.625,0.555556,0.5
3100,0.0,0.000000,1.25,1,0.833333,0.714286,0.625,0.555556,0.5
3200,2.5,1.666667,1.25,1,0.833333,0.714286,0.625,0.555556,1.0
3300,2.5,1.666667,1.25,1,0.833333,0.714286,1.250,1.111111,1.0
3400,0.0,1.666667,1.25,1,1.666667,1.428571,1.250,1.666667,1.5
3500,0.0,0.000000,1.25,2,1.666667,2.142857,1.875,1.666667,1.5
3600,2.5,1.666667,1.25,2,1.666667,2.142857,1.875,2.222222,2.5
3700,2.5,3.333333,2.50,2,1.666667,2.142857,2.500,3.333333,3.0
3800,2.5,3.333333,2.50,3,3.333333,3.571429,3.125,3.333333,3.0
3900,2.5,3.333333,3.75,5,4.166667,4.285714,3.750,3.333333,3.0


In [38]:
pdRythmesVitesses=pd.merge(timeRythmes,timeVitesses,left_index=True,right_index=True)
pdRythmesVitesses

,RJ1.00,RJ1.50,RJ2.00,RJ2.50,RJ3.00,RJ3.50,RJ4.00,RJ4.50,RJ5.00,RC1.00,RC1.50,RC2.00,RC2.50,RC3.00,RC3.50,RC4.00,RC4.50,RC5.00
3000,0.0,0.000000,0.00,0,0.833333,0.714286,0.625,0.555556,0.5,0.0,0.000000,0.00,0,0.833333,0.714286,0.625,0.555556,0.5
3100,0.0,0.000000,1.25,1,0.833333,0.714286,0.625,0.555556,0.5,0.0,0.000000,1.25,1,0.833333,0.714286,0.625,0.555556,0.5
3200,2.5,1.666667,1.25,1,0.833333,0.714286,0.625,0.555556,1.0,2.5,1.666667,1.25,1,0.833333,0.714286,0.625,0.555556,0.5
3300,2.5,1.666667,1.25,1,0.833333,0.714286,1.250,1.111111,1.0,2.5,1.666667,1.25,1,0.833333,0.714286,0.625,0.555556,0.5
3400,0.0,1.666667,1.25,1,1.666667,1.428571,1.250,1.666667,1.5,0.0,1.666667,1.25,1,0.833333,0.714286,0.625,0.555556,0.5
3500,0.0,0.000000,1.25,2,1.666667,2.142857,1.875,1.666667,1.5,0.0,0.000000,0.00,1,0.833333,0.714286,0.625,0.555556,0.5
3600,2.5,1.666667,1.25,2,1.666667,2.142857,1.875,2.222222,2.5,0.0,0.000000,0.00,0,0.000000,0.714286,0.625,0.555556,1.0
3700,2.5,3.333333,2.50,2,1.666667,2.142857,2.500,3.333333,3.0,0.0,0.000000,0.00,0,0.000000,0.000000,0.625,1.111111,1.0
3800,2.5,3.333333,2.50,3,3.333333,3.571429,3.125,3.333333,3.0,0.0,0.000000,0.00,0,0.833333,0.714286,0.625,0.555556,0.5
3900,2.5,3.333333,3.75,5,4.166667,4.285714,3.750,3.333333,3.0,0.0,0.000000,1.25,1,0.833333,0.714286,0.625,0.555556,0.5


In [39]:
with codecs.open(rythmeName.replace(".csv","-Pandas"+".pkl"),"wb") as output:
    pickle.dump(pdRythmesVitesses, output, pickle.HIGHEST_PROTOCOL)


In [40]:
if 0:
    with codecs.open(rythmeName.replace(".csv","-Total"+".csv"),"w",encoding="utf8") as output:
        output.write("time;%s"%";".join(["%.2f"%i for i in demiFenetres])+"\n")
        for element in sorted(resultats):       
            output.write("%.3f;%s"%(element,";".join(["%.1f"%i for i in resultats[element]]))+"\n")
